# Install all of the Dependency

In [31]:
# !pip install openai yfinance duckduckgo-search phidata
# !pip install groq
# !pip install fastapi uvicorn
# !pip install python-dotenv
# !pip install -U googlesearch-python pycountry
# !pip install crawlbase
# !pip install praw
#!pip install fastapi uvicorn



# Create Web Search Agent and used GOOGLE & DUCKDUCKGO Serch Engines

In [12]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.googlesearch import GoogleSearch

from dotenv import load_dotenv

load_dotenv()

web_search_agent = Agent(
    name = "Web Agent",
    description = "You are an advanced AI research assistant specializing in comprehensive and detailed research.",
    model = Groq(id="llama-3.3-70b-versatile"),
    tools  = [GoogleSearch(),DuckDuckGo()],
    instructions = """Your task is to gather factual and data-driven insights about any topic or product/service, along with its similar offerings, adhering to the following guidelines:

Conduct thorough research to gather as much relevant information as possible about the target product/service, its features, use cases, and benefits.
Identify and document similar products/services, their details, pricing, features, and differentiating factors.
Search for and include relevant URLs, articles, and official sources to back up the findings.
After gathering initial data, evaluate whether further searches based on collected information are necessary to improve the depth and quality of the research. Continue this refinement process for up to 3 iterations if needed.
Ensure all insights are factual, accurate, and well-supported by references. Avoid making any assumptions or generating speculative content.
Organize the output systematically, including:
A detailed summary of the target product/service.
Key features, pricing, benefits, and applications.
Comparative analysis with similar offerings, highlighting similarities and differences.
A list of reference links and articles used to compile the research.
Final Output Structure:

Overview of the Target Product/Service:

[Details about the product/service]
Similar Products/Services provided by the organizations:

[Details and comparative analysis]
Key Insights and Findings:

[Summarized points]
References and Data Sources:

[Links and sources]
Follow these steps rigorously to ensure the most comprehensive and accurate research possible. """,
    show_tool_calls = True,
    markdown = True,
    debug_mode=False
)
topic = input("Enter the topic you want to search: ")
web_search_agent.print_response(topic, stream=True)


Enter the topic you want to search: samsung mobile 


Output()

# CRUNCHBASE AGENT-

1.   Create Crunchbase Site Scraper and use it in agent tools
2.   Crunchbase agent for organizations details analysis



In [14]:
from crawlbase import CrawlingAPI
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

def crawl(organizations=None):
    if organizations is None or not isinstance(organizations, list):
        raise ValueError("organizations parameter must be a list of organization names")

    # Get API token from environment variable
    api_token = os.getenv('CRAWLBASE_API_TOKEN')
    if not api_token:
        raise ValueError("CRAWLBASE_API_TOKEN not found in environment variables")

    # Initialize the CrawlingAPI object with your token
    api = CrawlingAPI({'token': api_token})

    # Handle both single and multiple organizations
    if len(organizations) == 1:
        # Single organization case
        org = organizations[0]
        page_url = f'https://www.crunchbase.com/organization/{org}'
        response = api.get(page_url)

        if response['status_code'] == 200:
            scraped_data = scrape_data(response)
            print(f'Data: {scraped_data}')
            return scraped_data
        else:
            print(f"Error: {response}")
            return None
    else:
        # Multiple organizations case
        results = {}
        for org in organizations:
            page_url = f'https://www.crunchbase.com/organization/{org}'
            response = api.get(page_url)

            if response['status_code'] == 200:
                scraped_data = scrape_data(response)
                results[org] = scraped_data
                print(f'Data for {org}: {scraped_data}')
            else:
                print(f"Error for {org}: {response}")
                results[org] = None

        return results

def scrape_data(response):
    try:
        # Parse the HTML content using Beautiful Soup
        soup = BeautifulSoup(response['body'], 'html.parser')

        # Extract the title of the Crunchbase page
        title = soup.find('h1', class_='profile-name').get_text(strip=True)

        # Extract the description of the Crunchbase page
        description = soup.find('span', class_='description').get_text(strip=True)

        # Extract the location of the Crunchbase page
        location = soup.select_one('.section-content-wrapper li.ng-star-inserted').text.strip()

        # Extract the employees of the Crunchbase page
        employees = soup.select_one('.section-content-wrapper li.ng-star-inserted:nth-of-type(2)').text.strip()

        # Extract the Company URL of the Crunchbase page
        company_url = soup.select_one('.section-content-wrapper li.ng-star-inserted:nth-of-type(5) a[role="link"]')['href']

        # Extract the Rank of the Crunchbase page
        rank = soup.select_one('.section-content-wrapper li.ng-star-inserted:nth-of-type(6) span').text.strip()

        # Extract the Founded Date of the Crunchbase page
        founded = soup.select_one('.mat-mdc-card.mdc-card .text_and_value li:nth-of-type(4) field-formatter').text.strip()

        # Extract the Founders of the Crunchbase page
        founders = soup.select_one('.mat-mdc-card.mdc-card .text_and_value li:nth-of-type(5) field-formatter').text.strip()

        return {
            'title': title,
            'description': description,
            'location': location,
            'employees': employees,
            'company_url': company_url,
            'rank': rank,
            'founded': founded,
            'founders': founders,
        }
    except Exception as e:
        print(f"An error occurred: {e}")
        return {}

# if __name__ == "__main__":
#     # Example usage with list parameter
#     # For single organization
#     # single_result = crawl(organizations=['openai'])
#     # print("\nSingle organization result:", single_result)

#     # For multiple organizations
#     multiple_results = crawl(organizations=['openai', 'google'])
#     print("\nMultiple organizations results:", multiple_results)

In [15]:
import json
import httpx
crunch_agent = Agent(
    tools=[crawl],
    description = "You are an advanced AI research assistant specializing in comprehensive and detailed research.",
    model = Groq(id="llama-3.3-70b-versatile"),
    instructions = """ example of use tools is crawl(organizations=['openai', 'google']) get organizations informations like
            'title': title,
            'description': description,
            'location': location,
            'employees': employees,
            'company_url': company_url,
            'rank': rank,
            'founded': founded,
            'founders': founders,
          from tools and make a proper details report.
    """,
    show_tool_calls=True,
    markdown=True)
crunch_agent.print_response("organizations details report", stream=True)

Output()

WARNING  Could not run function crawl(organizations=['openai', 'google'])

ERROR    CRAWLBASE_API_TOKEN not found in environment variables                                                    
         Traceback (most recent call last):                                                                        
           File "/usr/local/lib/python3.10/dist-packages/phi/tools/function.py", line 319, in execute              
             self.result = self.function.entrypoint(**entrypoint_args, **self.arguments)                           
           File "/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_validate_call.py", line 38, in        
         wrapper_function                                                                                          
             return wrapper(*args, **kwargs)                                                                       
           File "/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_validate_call.py", line 111, in       
         __call__                                                                                                  
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
           File "<ipython-input-14-72a807de8ed5>", line 16, in crawl                                               
             raise ValueError("CRAWLBASE_API_TOKEN not found in environment variables")                            
         ValueError: CRAWLBASE_API_TOKEN not found in environment variables

# REDDIT AGENT -

1.   Create Reddit Scaraper to get user post and comments
2.   analyze this post and comments through agents



In [5]:
# reddit_scraper.py
import praw
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import datetime

def scrape_subreddit(subreddits, post_limit=3, comment_limit=2):
    """
    Scrape posts and comments from multiple subreddits.

    Parameters:
    subreddits (list or str): Single subreddit name or list of subreddit names to scrape
    post_limit (int): Number of top posts to scrape per subreddit (default: 10)
    comment_limit (int): Number of comment levels to replace (default: 5)

    Returns:
    pandas.DataFrame: DataFrame containing posts and comments data from all subreddits
    """
    # Convert single subreddit to list for consistent handling
    if isinstance(subreddits, str):
        subreddits = [subreddits]

    # Load environment variables
    load_dotenv()

    # Initialize Reddit instance using environment variables
    reddit = praw.Reddit(
        client_id=os.getenv('REDDIT_CLIENT_ID'),
        client_secret=os.getenv('REDDIT_CLIENT_SECRET'),
        user_agent=os.getenv('REDDIT_USER_AGENT'),
        check_for_async=False
    )

    data = []

    # Scrape each subreddit
    for subreddit_name in subreddits:
        try:
            print(f"Scraping r/{subreddit_name}...")
            subreddit = reddit.subreddit(subreddit_name)

            # Scrape posts and comments
            for post in subreddit.top(limit=post_limit):
                # Add post data
                data.append({
                    'Subreddit': subreddit_name,
                    'Type': 'Post',
                    'Post_id': post.id,
                    'Title': post.title,
                    'Author': post.author.name if post.author else 'Unknown',
                    'Timestamp': datetime.fromtimestamp(post.created_utc),
                    'Text': post.selftext,
                    'Score': post.score,
                    'Total_comments': post.num_comments,
                    'Post_URL': post.url
                })

                # Scrape comments if they exist
                if post.num_comments > 0:
                    post.comments.replace_more(limit=comment_limit)
                    for comment in post.comments.list():
                        data.append({
                            'Subreddit': subreddit_name,
                            'Type': 'Comment',
                            'Post_id': post.id,
                            'Title': post.title,
                            'Author': comment.author.name if comment.author else 'Unknown',
                            'Timestamp': datetime.fromtimestamp(comment.created_utc),
                            'Text': comment.body,
                            'Score': comment.score,
                            'Total_comments': 0,
                            'Post_URL': None
                        })

        except Exception as e:
            print(f"Error scraping r/{subreddit_name}: {str(e)}")
            continue

    if not data:
        print("No data was collected from any subreddit")
        return None

    df = pd.DataFrame(data)
    print(f"Successfully scraped {len(df)} posts and comments from {len(subreddits)} subreddits")
    return df

# Example usage:
# if __name__ == "__main__":
#     # Example with multiple subreddits
#     subreddits_to_scrape = ['fitness', 'technology']
#     df = scrape_subreddit(subreddits_to_scrape)
#     if df is not None:
#         df.to_csv('reddit_data.csv', index=False)

Scraping r/fitness...
Scraping r/technology...
Successfully scraped 3320 posts and comments from 2 subreddits


In [17]:
import json
import httpx
reddit_agent = Agent(
    tools=[scrape_subreddit],
    description = "You are an advanced AI research assistant specializing in comprehensive and detailed research.",
    model = Groq(id="llama-3.3-70b-versatile"),
    instructions = """ Use tools like subreddits_to_scrape = ['openai', 'google'] to gather organization names.
     Analyze consumer sentiment and trustworthiness for each organization. Study user perspectives regarding their services, and generate a detailed report with insights organized by organization.
    """,
    show_tool_calls=True,
    markdown=True)
reddit_agent.print_response("organizations wise sentiment analysis report", stream=True)

Output()

Scraping r/openai...

Scraping r/google...

Successfully scraped 49803 posts and comments from 2 subreddits

BadRequestError: Error code: 400 - {'error': {'message': "'messages.3' : for 'role:tool' the following must be satisfied[('messages.3.content' : value must be a string)]", 'type': 'invalid_request_error'}}

# Summarizer AGENT

In [18]:
# Define summarizer Agent
summary_writer = Agent(
    name="Trend Analyzer",
    role="Analyzes competitor organizations data",
    model = Groq(id="llama-3.3-70b-versatile"),
    instructions="""
    Analyze the competitor data and provide strategic insights:
    Provide insights on:
        1. Market positioning
        2. Pricing strategy
        3. Product features
        4. Marketing approach
        5. Recommended counter-strategies
    """,
    show_tool_calls=True,
    markdown=True,
)

# MULTI AGENT WORKFLOW

In [26]:
# The multi agent Team setup of phidata:
agent_team = Agent(
    agents=[web_search_agent, crunch_agent, reddit_agent, summary_writer],
    model = Groq(id="llama-3.3-70b-versatile"),
    instructions=[
        "First, web search for product or company details & similer product or services provided by other organizations details informations related to the user's specified topic.",
        "Then, provide the collected organizations name to the crunch agent.",
        "Important: you must ensure that the crunch agent receives all the organizations name to read.",
        "Next, the crunch agent will read the data and prepare details report of each.",
        "After reporting the informations will be passed to the summary_writer as well as reddit agent will read the organizations user sentimant and the informations will be passed to the summary_writer.",
        "Finally, the summary writter will identify emerging trends and potential business stratagy and compititor market as well as startup opportunities based on the summaries provided in a detailed Report form and create a proper report for make business decisions"
    ],
    show_tool_calls=True,
    markdown=True,
)

# Executing the workflow
# Step 1: Collect information
search = web_search_agent.run(f"Collect details informations from the user provided {topic}")
informations = search.content

# Step 2: competitor info
crunch = crunch_agent.run(f"create details report for the organizations name wise:\n{informations}")
company_info = crunch.content

# Step 3: Analyze trends
reddit_response = reddit_agent.run(f"Analyze business and user sentiment regarding the business from the following summaries:\n{informations}")
analysis = reddit_response.content

# Step 3: Analyze trends
trend_response = summary_writer.run(f"Analyze competitor and business from the following details:\n{company_info}, {analysis}")
report = trend_response.content

An error occurred: 'NoneType' object has no attribute 'get_text'
Data for Samsung Electronics: {}
An error occurred: 'NoneType' object has no attribute 'get_text'
Data for Apple Inc.: {}
Error for Google LLC: {'headers': {}, 'body': '', 'status_code': 520}
An error occurred: 'NoneType' object has no attribute 'get_text'
Data for OnePlus: {}
An error occurred: 'NoneType' object has no attribute 'get_text'
Data for Huawei Technologies: {}
An error occurred: 'NoneType' object has no attribute 'get_text'
Data for Xiaomi: {}


ValidationError: 2 validation errors for Message
content.list[any]
  Input should be a valid list [type=list_type, input_value={'Samsung Electronics': {...gies': {}, 'Xiaomi': {}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/list_type
content.str
  Input should be a valid string [type=string_type, input_value={'Samsung Electronics': {...gies': {}, 'Xiaomi': {}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

# **COMPETITIOR ANALYSIS FINAL REPORT**

In [27]:
agent_team.print_response("organizations wise details report", stream=True)

▰▱▱▱▱▱▱ Thinking...
┏━ Message ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃ organizations wise details report                                                                               ┃
┃                                                                                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
┏━ Response (4.4s) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                                                                                 ┃
┃                                        Organizations Wise Details Report                                        ┃
┃                                                                                                                 ┃
┃ Based on the user's specified topic, the following organizations were found to be relevant:                     ┃
┃                                                                                                                 ┃
┃                                                    1. Amazon                                                    ┃
┃                                                                                                                 ┃
┃  • Product/Service: E-commerce platform, cloud computing, artificial intelligence, and more                     ┃
┃  • Description: Amazon is a multinational technology company that offers a wide range of products and services, ┃
┃    including online shopping, cloud computing, and artificial intelligence.                                     ┃
┃  • Competitors: Walmart, eBay, Google, Microsoft                                                                ┃
┃                                                                                                                 ┃
┃                                                    2. Google                                                    ┃
┃                                                                                                                 ┃
┃  • Product/Service: Search engine, online advertising, cloud computing, and more                                ┃
┃  • Description: Google is a multinational technology company that specializes in internet-related services and  ┃
┃    products, including search, online advertising, and cloud computing.                                         ┃
┃  • Competitors: Microsoft, Amazon, Facebook, Yahoo                                                              ┃
┃                                                                                                                 ┃
┃                                                  3. Microsoft                                                   ┃
┃                                                                                                                 ┃
┃  • Product/Service: Software, cloud computing, gaming, and more                                                 ┃
┃  • Description: Microsoft is a multinational technology company that develops, manufactures, and sells computer ┃
┃    software, consumer electronics, and personal computers.                                                      ┃
┃  • Competitors: Google, Amazon, Apple, Oracle                                                                   ┃
┃                                                                                                                 ┃
┃                                                   4. Facebook                                                   ┃
┃                                                                                                                 ┃
┃  • Product/Service: Social media p